<h1>IDEA: Un algoritmo de clave privada</h1>

<h2> <u>Cifrado de un mensaje, dados los siguientes datos:</u></h2>
<li> Mensaje="En un lugar de la Mancha, de cuyo nombre no quiero acordarme"</li>
<li> Clave privada (K) => 128 bits ="MARCOSDIEGOSANTI"

<h2>1. Obtención de los valores binarios del mensaje y la clave </h2>

In [18]:
def leerFichero(ruta):
    fichero = open(ruta, "r", encoding="utf8")
    texto = fichero.read()
    fichero.close()
    return texto

In [19]:
mensaje_claro="En un lugar de la Mancha, de cuyo nombre no quiero acordarme"
#La clave tiene que tener 128 bits
clave_privada=leerFichero("clavePrivadaDescifrada.txt")
#ⵁ -> Caracter reservado
alf=leerFichero("alf_idea.txt")
print("Clave privada ->",clave_privada)
print("Alfabeto ->",alf)


Clave privada -> TRABAJOEXCELENTE
Alfabeto -> AÁBCDEÉFGHIÍJKLMNÑOÓPQRSTUÚVWXYZ.,;:()¿?¡!-0123456789aábcdeéfghiíjklmnñoópqrstuúvwxyz €$&㎀㎁㎂㎃㎄㎅㎆㎇㎈㎉㎊㎋㎌㎍㎎㎏㎐㎑㎒㎓㎔㎕㎖㎗㎘㎙㎚㎛㎜㎝㎞㎟㎠㎡㎢㎣㎤㎥㎦㎧㎨㎩㎪㎫㎬㎭㎮㎯㎰㎱㎲㎳㎴㎵㎶㎷㎸㎹㎺㎻㎼㎽㎾㎿㏀㏁㏂㏃㏄㏅㏆㏇㏈㏉㏊㏋㏌㏍㏎㏏㏐㏑㏒㏓㏔㏕㏖㏗㏘㏙㏚㏛㏜㏝㍱㍲㍳㍴㍵㍶㍷㍸㍹䷀䷁䷂䷃䷄䷅䷆䷇䷈䷉䷊䷋䷌䷍䷎䷏䷐䷑䷒䷓䷔䷕䷖䷗䷘䷙䷚䷛䷜䷝䷞䷟䷠䷡䷢䷣䷤䷥䷦䷧䷨䷩䷪䷫䷬䷭䷮䷯䷰䷱䷲䷳䷴䷵䷶䷷䷸䷹䷺䷻䷼䷽ⵁ


In [20]:
def cadena_a_binario(cadena):
    #Conversion a binario de 8 bits
    cadena_binaria = "".join(format(alf.index(x), '08b') for x in cadena)
    return cadena_binaria

In [21]:
#Conversión a binario de la clave y el mensaje. Cada carácter representado en binario de 8 bits
mensaje_binario = cadena_a_binario(mensaje_claro)
clave_binaria = cadena_a_binario(clave_privada)

<h2>2. Se divide el mensaje binario en bloques de 64 bits. Cada bloque se divide a su vez en 4 sub-bloques de 16 bits </h2>


In [22]:
def binario_a_bloques(cadena, tam_bloque):
    lista_bloques = [cadena[x:x+tam_bloque] for x in range(0,len(cadena), tam_bloque)]
    len_ultimo_bloque = len(lista_bloques[-1])
    #Relleno con 0s si no se alcanza el tamaño del bloque
    if(len_ultimo_bloque!=tam_bloque):
        #Relleno con ⵁ hasta alcanzar el tamaño del bloque
        lista_bloques[-1] =  lista_bloques[-1] + format(alf.index("ⵁ"), '08b') * ((tam_bloque-len_ultimo_bloque)//8)
    return lista_bloques


In [23]:
def procesar_mensaje_binario(mensaje_binario):

    #Mensaje dividido en bloques de 64 bits, que tendrá la forma [ab,cd,ef,gh]
    mensaje_dividido = binario_a_bloques(mensaje_binario, 64)

    #Cada sub-bloque lo divido en 4 bloques de 16 bits, guardo el resultado final, que tendrá la forma [[a,b],[c,d],[e,f],[g,h]]
    mensaje_bloques = []
    for bloque in mensaje_dividido:
        mensaje_bloques.append(binario_a_bloques(bloque, 16))
    return mensaje_bloques

In [24]:
#Mensaje dividido en sub-bloques
mensaje_bloques = procesar_mensaje_binario(mensaje_binario)

<h2>3. Se calculan las claves intermedias. Serán un total de 52 subclaves de 16 bits, que se emplearán para cifrar el mensaje</h2>

In [25]:
 #Recibe un string de valores binarios
 def generar_claves_intermedias_cifrado(clave_inicial):
     # lista de una dimensión, que contendrá las 52 subclaves
     lista_claves = []
     #La agrupación será de 6 en 6 subclaves. Las 8 primeras son la clave inicial.
     lista_claves.extend(binario_a_bloques(clave_inicial, 16))

     while(len(lista_claves)<52):
         #Desplazar 25 bits a la izquierda la clave inicial
         clave_inicial = clave_inicial[25:]+clave_inicial[:25]
         #Bloques de la clave actualizada
         lista_claves.extend(binario_a_bloques(clave_inicial, 16))

     # lista_claves crece en múltiplos de 8 y es por ello que debemos quedarnos solo con los 52 primeros valores
     lista_claves = lista_claves[:52]

     return lista_claves

In [26]:
#Claves intermedias para cifrar
claves_intermedias = generar_claves_intermedias_cifrado(clave_binaria)

<h2>4. Aplico el algoritmo de cifrado a cada bloque del mensaje</h2>

In [27]:
def calcular_iteracion(bloque_numerico, subclaves):
        #Paso 1
        var1 = (bloque_numerico[0] * subclaves[0])%(2**16+1)

        #Paso 2
        var2 = (bloque_numerico[1] + subclaves[1])%(2**16)

        #Paso 3
        var3 = (bloque_numerico[2] + subclaves[2])%(2**16)

        #Paso 4
        var4 = (bloque_numerico[3] * subclaves[3])%(2**16+1)

        #Paso 5 (XOR entre 1 y 3)
        var5 = var1 ^ var3

        #Paso 6 
        var6 = var2 ^ var4

        #Paso 7
        var7 = (var5 * subclaves[4])%(2**16+1)

        #Paso 8
        var8 = (var6 + var7)%(2**16)

        #Paso 9
        var9 = (var8 * subclaves[5])%(2**16+1)

        #Paso 10
        var10 = (var7 + var9)%(2**16)

        #Paso 11
        var11 = var1 ^ var9

        #Paso 12
        var12 = var3 ^ var9

        #Paso 13
        var13 = var2 ^ var10

        #Paso 14
        var14 = var4 ^ var10

        return [var11, var12, var13, var14]


In [28]:
def lista_binaria_a_decimal (lista_binaria):
    return [int(elem,2) for elem in lista_binaria]

In [29]:
def bloque_16_a_texto(bloque):
    pos1 = int(bloque[0:8], 2)
    pos2 = int(bloque[8:17], 2)
    #print(pos1, pos2)
    cadena = alf[pos1] + alf[pos2]
    return cadena


In [30]:
def aplicar_cifrado_bloque(bloque_msj, claves_intermedias):
    #La primera iteración usara los valores numericos del bloque del mensaje
    resultado_iteracion = lista_binaria_a_decimal(bloque_msj)
    #resultado_iteracion = [int(bloque_msj[0],2), int(bloque_msj[1],2), int(bloque_msj[2],2), int(bloque_msj[3],2)]
    iteraciones = []

    for i in range(0,8):
        #En cada iteración se cogen 6 subclaves
        subclaves = lista_binaria_a_decimal(claves_intermedias[i*6:(i*6)+6])
        resultado_iteracion = calcular_iteracion(resultado_iteracion, subclaves)
        iteraciones.extend(resultado_iteracion)
        #Se retroalimenta el algoritmo usando como bloque de entrada la permutación de los elementos centrales de la anterior iteración
        resultado_iteracion = [resultado_iteracion[0], resultado_iteracion[1], resultado_iteracion[2], resultado_iteracion[3]]

    #Último paso 
    #Ultimas 4 subclaves
    subclaves = lista_binaria_a_decimal(claves_intermedias[48:52])

    resultado_iteracion = [resultado_iteracion[0], resultado_iteracion[2], resultado_iteracion[1], resultado_iteracion[3]]
    var15 = (resultado_iteracion[0] * subclaves[0])%(2**16+1)
    var16 = (resultado_iteracion[1] + subclaves[1])%(2**16)
    var17 = (resultado_iteracion[2] + subclaves[2])%(2**16)
    var18 = (resultado_iteracion[3] * subclaves[3])%(2**16+1)

    resultado = [var15, var16, var17, var18]

    return resultado

In [31]:
def cifrar_descifrar_mensaje(mensaje_bloques, claves_intermedias):
    msj_cifrado_decimal = []

    for bloque in mensaje_bloques:
        msj_cifrado_decimal.extend(aplicar_cifrado_bloque(bloque, claves_intermedias))

    msj_cifrado_binario = [format(elem, "016b") for elem in msj_cifrado_decimal]

    mensaje_cifrado = ""

    for elem in msj_cifrado_binario:
        mensaje_cifrado += bloque_16_a_texto(elem)

    return mensaje_cifrado

In [32]:
#Obtención del mensaje cifrado
msj_cifrado = cifrar_descifrar_mensaje(mensaje_bloques, claves_intermedias)

print("Mensaje cifrado ->",msj_cifrado)

Mensaje cifrado -> ䷮䷷㏚㏙㏀㎰㏍䷨㏚䷲䷢c㏑㏄䷑!$㏂㍷B䷋㎚㎬㎪Qií㎔jDn6㏔P䷃䷼㎢㏇t䷢䷭㎀k㎍㏏3Q㍸䷜m䷅㎺㍷8㎮㍸䷚㎶Ó䷂㎒:㎝Y


<h2><u>Descifrado de un mensaje, dados los siguientes datos:</u></h2>

<h2>1. Obtener las subclaves de descifrado </h2>

In [33]:
 #Recibe un string de valores binarios
 def generar_claves_intermedias_descifrado(clave_inicial):
    # lista de una dimensión, que contendrá las 52 subclaves
    lista_claves_binaria = generar_claves_intermedias_cifrado(clave_inicial)
    lista_claves_decimal = lista_binaria_a_decimal(lista_claves_binaria)
    i = 0

    while(i<52):
         if(i%3 == 0):
            #Calculo el inverso modulo de la multiplicación
            lista_claves_decimal[i] = pow(lista_claves_decimal[i], -1, (2**16+1))
         else:
            #No aplico nada a los dos ultimos valores de cada fila
            if(not((i+1)%6==5 or (i)%6==5)):
                #Calculo el valor negativo modulo de la suma
                lista_claves_decimal[i] = (lista_claves_decimal[i]*-1)%(2**16)
                
         i=i+1

    #Ordenación de claves de descifrado según algoritmo
    lista_claves_descifrado = []

    lista_claves_descifrado.extend(lista_claves_decimal[48:52])
    lista_claves_descifrado.extend(lista_claves_decimal[46:48])
    
    for i in range (42,0,-6):
        lista_claves_descifrado.append(lista_claves_decimal[i])
        lista_claves_descifrado.append(lista_claves_decimal[i+2])
        lista_claves_descifrado.append(lista_claves_decimal[i+1])
        lista_claves_descifrado.append(lista_claves_decimal[i+3])
        lista_claves_descifrado.extend(lista_claves_decimal[i-2:i])

    lista_claves_descifrado.extend(lista_claves_decimal[0:4])

    #print_bloque_16_a_hex(lista_claves_descifrado)
    return [format(elem, "016b") for elem in lista_claves_descifrado]



In [34]:
msj_cifrado = "䷮䷷㏚㏙㏀㎰㏍䷨㏚䷲䷢c㏑㏄䷑!$㏂㍷B䷋㎚㎬㎪Qií㎔jDn6㏔P䷃䷼㎢㏇t䷢䷭㎀k㎍㏏3Q㍸䷜m䷅㎺㍷8㎮㍸䷚㎶Ó䷂㎒:㎝Y"

#Genero las claves de descifrado, partiendo de la clave publica compartida
claves_intermedias_descifrar = generar_claves_intermedias_descifrado(clave_binaria)

#Generación de sub-bloques del mensaje
msj_cifrado_procesado = procesar_mensaje_binario(cadena_a_binario(msj_cifrado))

#Obtención del mensaje en claro
msj_claro = cifrar_descifrar_mensaje(msj_cifrado_procesado, claves_intermedias_descifrar)

#Elimino el relleno
mensaje_claro = mensaje_claro.replace("ⵁ", "")

print("Mensaje descifrado ->",mensaje_claro)

Mensaje descifrado -> En un lugar de la Mancha, de cuyo nombre no quiero acordarme
